In [ ]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 37 kB/s 
     |████████████████████████████████| 199 kB 56.0 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=00502441cfd6990bf52915a697a20207c79b762c7a0de7d76af72cd18b565389
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark


In [ ]:
import pyspark

In [ ]:
from pyspark.sql import SparkSession

In [ ]:
spark=SparkSession.builder.appName("test").getOrCreate()

In [ ]:
spark

In [ ]:
df=spark.read.csv("/content/drive/MyDrive/data/house price pyspark/train.csv",header=True,inferSchema=True)

In [ ]:
df=df.drop("ADDRESS")

In [ ]:
df.show()

+---------+------------------+----+-------+---------+------------------+-------------+------+------------------+-----------------+---------------------+
|POSTED_BY|UNDER_CONSTRUCTION|RERA|BHK_NO.|BHK_OR_RK|         SQUARE_FT|READY_TO_MOVE|RESALE|         LONGITUDE|         LATITUDE|TARGET(PRICE_IN_LACS)|
+---------+------------------+----+-------+---------+------------------+-------------+------+------------------+-----------------+---------------------+
|    Owner|                 0|   0|      2|      BHK|       1300.236407|            1|     1|          12.96991|         77.59796|                 55.0|
|   Dealer|                 0|   0|      2|      BHK|            1275.0|            1|     1|         12.274538|        76.644605|                 51.0|
|    Owner|                 0|   0|      2|      BHK| 933.1597222000001|            1|     1|         12.778033|        77.632191|                 43.0|
|    Owner|                 0|   1|      2|      BHK| 929.9211427000001|          

In [ ]:
type(df)

pyspark.sql.dataframe.DataFrame

In [ ]:
df.printSchema()

root
 |-- POSTED_BY: string (nullable = true)
 |-- UNDER_CONSTRUCTION: integer (nullable = true)
 |-- RERA: integer (nullable = true)
 |-- BHK_NO.: integer (nullable = true)
 |-- BHK_OR_RK: string (nullable = true)
 |-- SQUARE_FT: double (nullable = true)
 |-- READY_TO_MOVE: integer (nullable = true)
 |-- RESALE: integer (nullable = true)
 |-- LONGITUDE: double (nullable = true)
 |-- LATITUDE: double (nullable = true)
 |-- TARGET(PRICE_IN_LACS): double (nullable = true)



In [ ]:
df.groupBy("POSTED_BY").count().show()

+---------+-----+
|POSTED_BY|count|
+---------+-----+
|  Builder|  622|
|   Dealer|18291|
|    Owner|10538|
+---------+-----+



In [ ]:
df.groupBy("BHK_OR_RK").count().show()

+---------+-----+
|BHK_OR_RK|count|
+---------+-----+
|      BHK|29427|
|       RK|   24|
+---------+-----+



In [ ]:
df=df.drop("BHK_OR_RK")
df=df.drop("READY_TO_MOVE")

In [ ]:
df.groupBy("UNDER_CONSTRUCTION").count().show()

+------------------+-----+
|UNDER_CONSTRUCTION|count|
+------------------+-----+
|                 1| 5294|
|                 0|24157|
+------------------+-----+



In [ ]:
#RERA, the full form of which is Real Estate Regulatory Authority, stands for transparency in the real estate industry.
df.groupBy("RERA").count().show()

+----+-----+
|RERA|count|
+----+-----+
|   1| 9363|
|   0|20088|
+----+-----+



In [ ]:
df.groupBy("RESALE").count().show()

+------+-----+
|RESALE|count|
+------+-----+
|     1|27377|
|     0| 2074|
+------+-----+



In [ ]:
df.groupBy("`BHK_NO.`").count().show()

+-------+-----+
|BHK_NO.|count|
+-------+-----+
|     12|    3|
|      1| 3574|
|     13|    1|
|      6|   52|
|      3|10546|
|     20|    4|
|      5|  190|
|     15|    4|
|      9|    3|
|     17|    1|
|      4| 1723|
|      8|   10|
|      7|   11|
|     10|    4|
|     11|    1|
|      2|13324|
+-------+-----+



In [ ]:
df=df.withColumnRenamed('BHK_NO.', "BHK")

In [ ]:
df.groupBy("BHK").count().show()

+---+-----+
|BHK|count|
+---+-----+
| 12|    3|
|  1| 3574|
| 13|    1|
|  6|   52|
|  3|10546|
| 20|    4|
|  5|  190|
| 15|    4|
|  9|    3|
| 17|    1|
|  4| 1723|
|  8|   10|
|  7|   11|
| 10|    4|
| 11|    1|
|  2|13324|
+---+-----+



In [ ]:
df.show()

+---------+------------------+----+---+------------------+------+------------------+-----------------+---------------------+
|POSTED_BY|UNDER_CONSTRUCTION|RERA|BHK|         SQUARE_FT|RESALE|         LONGITUDE|         LATITUDE|TARGET(PRICE_IN_LACS)|
+---------+------------------+----+---+------------------+------+------------------+-----------------+---------------------+
|    Owner|                 0|   0|  2|       1300.236407|     1|          12.96991|         77.59796|                 55.0|
|   Dealer|                 0|   0|  2|            1275.0|     1|         12.274538|        76.644605|                 51.0|
|    Owner|                 0|   0|  2| 933.1597222000001|     1|         12.778033|        77.632191|                 43.0|
|    Owner|                 0|   1|  2| 929.9211427000001|     1|           28.6423|          77.3445|                 62.5|
|   Dealer|                 1|   0|  2| 999.0092470000001|     1|           22.5922|        88.484911|                 60.5|


In [ ]:
from pyspark.ml.feature import StringIndexer

In [ ]:
indexer=StringIndexer(inputCol="POSTED_BY",outputCol="POSTED_BY_encoded")
df=indexer.fit(df).transform(df)

In [ ]:
df=df.drop("POSTED_BY")

In [ ]:
from pyspark.ml.feature import VectorAssembler

In [ ]:
featureassembler=VectorAssembler(inputCols=["UNDER_CONSTRUCTION","BHK","POSTED_BY_encoded","RERA","SQUARE_FT","RESALE","LONGITUDE","LATITUDE"],outputCol="Indepedent Features")
result=featureassembler.transform(df)

In [ ]:
result.show()

+------------------+----+---+------------------+------+------------------+-----------------+---------------------+-----------------+--------------------+
|UNDER_CONSTRUCTION|RERA|BHK|         SQUARE_FT|RESALE|         LONGITUDE|         LATITUDE|TARGET(PRICE_IN_LACS)|POSTED_BY_encoded| Indepedent Features|
+------------------+----+---+------------------+------+------------------+-----------------+---------------------+-----------------+--------------------+
|                 0|   0|  2|       1300.236407|     1|          12.96991|         77.59796|                 55.0|              1.0|[0.0,2.0,1.0,0.0,...|
|                 0|   0|  2|            1275.0|     1|         12.274538|        76.644605|                 51.0|              0.0|[0.0,2.0,0.0,0.0,...|
|                 0|   0|  2| 933.1597222000001|     1|         12.778033|        77.632191|                 43.0|              1.0|[0.0,2.0,1.0,0.0,...|
|                 0|   1|  2| 929.9211427000001|     1|           28.6423|  

In [ ]:
result=result.select("Indepedent Features","TARGET(PRICE_IN_LACS)")

In [ ]:
result.show(truncate=0)

+---------------------------------------------------------------------+---------------------+
|Indepedent Features                                                  |TARGET(PRICE_IN_LACS)|
+---------------------------------------------------------------------+---------------------+
|[0.0,2.0,1.0,0.0,1300.236407,1.0,12.96991,77.59796]                  |55.0                 |
|[0.0,2.0,0.0,0.0,1275.0,1.0,12.274538,76.644605]                     |51.0                 |
|[0.0,2.0,1.0,0.0,933.1597222000001,1.0,12.778033,77.632191]          |43.0                 |
|[0.0,2.0,1.0,1.0,929.9211427000001,1.0,28.6423,77.3445]              |62.5                 |
|[1.0,2.0,0.0,0.0,999.0092470000001,1.0,22.5922,88.484911]            |60.5                 |
|[0.0,3.0,1.0,0.0,1250.0,1.0,10.03328,76.28257099999999]              |42.0                 |
|[0.0,3.0,0.0,0.0,1495.053957,1.0,26.916347,75.7956]                  |66.5                 |
|[0.0,3.0,1.0,1.0,1181.012946,1.0,30.74,76.65]              

In [ ]:
from pyspark.ml.regression import LinearRegression

In [ ]:
train_data,test_data=result.randomSplit([0.75,0.25])

In [ ]:
model=LinearRegression(featuresCol="Indepedent Features",labelCol="TARGET(PRICE_IN_LACS)")

In [ ]:
model=model.fit(train_data)

In [ ]:
model.coefficients

DenseVector([-40.6361, 81.4929, -114.4461, 5.3489, 0.0001, -584.5656, -4.4962, -1.4403])

In [ ]:
model.intercept

746.7017439458145

In [ ]:
pred=model.evaluate(test_data)

In [ ]:
print(pred.predictions.show(100,truncate=0))

+----------------------------------------------------------------------+---------------------+------------------+
|Indepedent Features                                                   |TARGET(PRICE_IN_LACS)|prediction        |
+----------------------------------------------------------------------+---------------------+------------------+
|(8,[1,4,6,7],[1.0,440.2920962,19.2167,73.0833])                       |41.0                 |636.5871484318246 |
|(8,[1,4,6,7],[1.0,717.4581483,29.892311,78.000723])                   |21.0                 |581.5401022840886 |
|(8,[1,4,6,7],[2.0,507.6142132,19.193770999999998,73.19789200000001])  |29.0                 |718.0266450920176 |
|(8,[1,4,6,7],[2.0,524.7619048,19.2167,73.0833])                       |55.1                 |718.0907851328693 |
|(8,[1,4,6,7],[2.0,610.8108107999999,12.78333,80.18333])               |22.6                 |736.8015173158597 |
|(8,[1,4,6,7],[2.0,62736.842110000005,13.0511,80.2125])                |5960.0          

In [ ]:
trainingSummary = model.summary

In [ ]:
print("RMSE: %f" % trainingSummary.rootMeanSquaredError)


RMSE: 580.798819
